<a href="https://colab.research.google.com/github/kerryback/data-portal-notebook/blob/main/Rice_Business_Data_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rice Business Stock Market Data - Python Example

This notebook demonstrates how to access and analyze stock market data from the Rice Business Data Portal using Python and DuckDB queries.

## 📋 What You'll Learn

- How to connect to Rice Business stock market data
- SQL queries with DuckDB syntax
- Stock analysis examples
- Data visualization with pandas and matplotlib

## 🔑 Getting Your Access Token

1. Visit the [Rice Business Data Portal](https://data-portal.rice-business.org)
2. Enter your `@rice.edu` email address
3. Check your email for the **access token**
4. Use that access token in the code below

**Note**: The access token is the same token you use to log into the web portal!

## 🔐 **Recommended: Store Your Token as a Secret in Google Colab**

For security, we recommend storing your access token as a **secret** in Google Colab instead of pasting it directly in the code:

### **How to Add a Secret:**
1. **Click the 🔑 key icon** on the left sidebar in Colab
2. **Click "Add new secret"**
3. **Name:** `RICE_ACCESS_TOKEN`
4. **Value:** Paste your access token from the email
5. **Enable notebook access** by toggling the switch

### **Benefits:**
- ✅ **Secure** - Token won't be visible in your notebook
- ✅ **Shareable** - You can share notebooks without exposing your token
- ✅ **Professional** - Best practice for sensitive credentials

If you don't set up a secret, you can still paste your token directly in the code below (look for `ACCESS_TOKEN = "YOUR_ACCESS_TOKEN_HERE"`).

---

## 🚀 Setup and Installation

First, let's install the required packages and download the Rice Data client.

In [ ]:
# Install required packages
!pip install requests pandas matplotlib seaborn plotly -q

print("✅ Packages installed successfully!")

In [ ]:
# Download the Rice Data Python client
import urllib.request
import os

# Download from GitHub repository
client_url = "https://raw.githubusercontent.com/kerryback/data-portal-notebook/main/rice_data_client.py"

try:
    urllib.request.urlretrieve(client_url, 'rice_data_client.py')
    print("✅ Rice Data client downloaded successfully!")
except Exception as e:
    print(f"❌ Download failed: {e}")
    print("💡 You can manually copy the client code from the Rice Business Data Portal")

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

## 🔐 Connect to Rice Business Data

**Replace `YOUR_ACCESS_TOKEN_HERE` with your actual access token from the Rice Business Data Portal.**

This is the same token you received via email and use to log into the web portal!

In [ ]:
# Import the Rice Data client
from rice_data_client import RiceDataClient

# 🔐 OPTION 1: Use Google Colab Secret (Recommended)
try:
    from google.colab import userdata
    ACCESS_TOKEN = userdata.get('RICE_ACCESS_TOKEN')
    print("✅ Using access token from Google Colab secrets")
except:
    # 🔑 OPTION 2: Paste your token directly (if you didn't set up a secret)
    ACCESS_TOKEN = "YOUR_ACCESS_TOKEN_HERE"  # Replace with your actual token
    if ACCESS_TOKEN == "YOUR_ACCESS_TOKEN_HERE":
        print("⚠️  Please either:")
        print("   1. Set up a secret named 'RICE_ACCESS_TOKEN' in Colab (recommended), or")
        print("   2. Replace 'YOUR_ACCESS_TOKEN_HERE' with your actual token")

# 🌐 Portal URL - Rice Business Data Portal
PORTAL_URL = "https://data-portal.rice-business.org"

# Connect to Rice Business data
try:
    client = RiceDataClient(
        access_token=ACCESS_TOKEN,
        base_url=PORTAL_URL
    )
    print("\n🎉 Successfully connected to Rice Business Stock Market Data!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("💡 Make sure you have:")
    print("   1. A valid access token from the Rice Business Data Portal")
    print("   2. Internet connection to reach data-portal.rice-business.org")
    print("   3. Your token hasn't expired (tokens are valid for 48 hours)")

## 📊 Explore Available Data

Let's start by exploring what data is available in our database.

In [ ]:
# Get available tables
tables = client.get_available_tables()
print("📋 Available Data Tables:")
for i, table in enumerate(tables, 1):
    print(f"   {i}. {table}")

# Get database statistics
stats = client.get_stats()
print(f"\n📈 Database Overview:")
print(f"   • Total companies: {stats['total_tickers']:,}")
print(f"   • Active companies: {stats['active_tickers']:,}")
print(f"   • Top exchanges: {', '.join([ex['exchange'] for ex in stats['top_exchanges'][:3]])}")

In [ ]:
# Explore sectors in the database
sectors = client.list_sectors()
print("🏢 Top 10 Sectors by Number of Companies:")
print(sectors.head(10).to_string(index=False))

# Create a simple visualization
plt.figure(figsize=(12, 6))
top_sectors = sectors.head(8)
plt.bar(range(len(top_sectors)), top_sectors['ticker_count'], color='steelblue')
plt.xlabel('Sector')
plt.ylabel('Number of Companies')
plt.title('Top 8 Sectors by Number of Companies')
plt.xticks(range(len(top_sectors)), top_sectors['sector'], rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 🔍 Basic DuckDB Queries

Now let's explore the data using SQL queries. DuckDB supports standard SQL syntax with some powerful extensions.

In [ ]:
# Example 1: Find technology companies
print("💻 Technology Companies (Sample):")
tech_companies = client.query("""
    SELECT ticker, name, industry, exchange, location
    FROM ndl.tickers 
    WHERE sector = 'Technology' 
    AND isdelisted = 'N'
    ORDER BY ticker
    LIMIT 10
""")

print(tech_companies.to_string(index=False))

In [ ]:
# Example 2: Get recent stock prices for major tech companies
print("📈 Recent Stock Prices for Major Tech Companies:")
recent_prices = client.query("""
    SELECT 
        ticker,
        date,
        close,
        volume,
        (close - open) / open * 100 as daily_return_pct
    FROM ndl.sep 
    WHERE ticker IN ('AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA')
    AND date >= '2024-01-01'
    ORDER BY ticker, date DESC
""")

print(f"Retrieved {len(recent_prices):,} price records")
print("\nSample data:")
print(recent_prices.head(10).to_string(index=False))

In [ ]:
# Example 3: Calculate moving averages using DuckDB window functions
print("📊 Moving Averages for Apple (AAPL):")
aapl_ma = client.query("""
    SELECT 
        date,
        close,
        AVG(close) OVER (
            ORDER BY date 
            ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
        ) as ma_10,
        AVG(close) OVER (
            ORDER BY date 
            ROWS BETWEEN 49 PRECEDING AND CURRENT ROW
        ) as ma_50
    FROM ndl.sep 
    WHERE ticker = 'AAPL'
    AND date >= '2024-01-01'
    ORDER BY date
""")

print(f"Calculated moving averages for {len(aapl_ma)} trading days")
print("\nRecent data:")
print(aapl_ma.tail().round(2).to_string(index=False))

## 📈 Advanced Analysis Examples

Let's perform some more sophisticated financial analysis using DuckDB's powerful features.

In [ ]:
# Example 4: Sector performance analysis
print("🏆 Sector Performance Analysis (YTD 2024):")
sector_performance = client.query("""
    WITH sector_returns AS (
        SELECT 
            t.sector,
            s.ticker,
            s.close as current_price,
            FIRST_VALUE(s.close) OVER (
                PARTITION BY s.ticker 
                ORDER BY s.date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) as start_price
        FROM ndl.sep s
        JOIN ndl.tickers t ON s.ticker = t.ticker
        WHERE s.date >= '2024-01-01'
        AND t.sector IS NOT NULL
        AND t.isdelisted = 'N'
        QUALIFY ROW_NUMBER() OVER (PARTITION BY s.ticker ORDER BY s.date DESC) = 1
    )
    SELECT 
        sector,
        COUNT(*) as num_companies,
        ROUND(AVG((current_price - start_price) / start_price * 100), 2) as avg_return_pct,
        ROUND(MEDIAN((current_price - start_price) / start_price * 100), 2) as median_return_pct
    FROM sector_returns
    WHERE start_price > 0 AND current_price > 0
    GROUP BY sector
    HAVING COUNT(*) >= 10
    ORDER BY avg_return_pct DESC
""")

print(sector_performance.to_string(index=False))

In [ ]:
# Visualize sector performance
plt.figure(figsize=(14, 8))
bars = plt.bar(range(len(sector_performance)), sector_performance['avg_return_pct'], 
               color=['green' if x > 0 else 'red' for x in sector_performance['avg_return_pct']])
plt.xlabel('Sector')
plt.ylabel('Average Return (%)')
plt.title('Sector Performance - Average YTD Returns (2024)')
plt.xticks(range(len(sector_performance)), sector_performance['sector'], rotation=45, ha='right')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5 if height > 0 else height - 1,
             f'{height:.1f}%', ha='center', va='bottom' if height > 0 else 'top')

plt.tight_layout()
plt.show()

In [ ]:
# Example 5: Find stocks with high trading volume
print("📊 High Volume Trading Analysis (Last 30 Days):")
high_volume = client.query("""
    SELECT 
        s.ticker,
        t.name,
        t.sector,
        ROUND(AVG(s.volume), 0) as avg_daily_volume,
        ROUND(AVG(s.close * s.volume), 0) as avg_daily_dollar_volume,
        COUNT(*) as trading_days
    FROM ndl.sep s
    JOIN ndl.tickers t ON s.ticker = t.ticker
    WHERE s.date >= CURRENT_DATE - INTERVAL '30 days'
    AND t.isdelisted = 'N'
    AND s.volume > 0
    GROUP BY s.ticker, t.name, t.sector
    HAVING COUNT(*) >= 20  -- At least 20 trading days
    ORDER BY avg_daily_dollar_volume DESC
    LIMIT 15
""")

print(high_volume.to_string(index=False))

## 💰 Fundamental Analysis

Let's explore financial fundamentals using the SF1 table with DuckDB queries.

In [ ]:
# Example 6: Financial ratios analysis
print("💰 Financial Ratios - Technology Sector Leaders:")
financial_ratios = client.query("""
    WITH latest_financials AS (
        SELECT 
            sf.ticker,
            t.name,
            sf.calendardate,
            sf.revenue,
            sf.netinc,
            sf.assets,
            sf.equity,
            sf.marketcap,
            sf.pe,
            sf.pb,
            ROW_NUMBER() OVER (PARTITION BY sf.ticker ORDER BY sf.calendardate DESC) as rn
        FROM ndl.sf1 sf
        JOIN ndl.tickers t ON sf.ticker = t.ticker
        WHERE t.sector = 'Technology'
        AND sf.dimension = 'ARY'  -- Annual data
        AND sf.calendardate >= '2023-01-01'
        AND t.isdelisted = 'N'
    )
    SELECT 
        ticker,
        name,
        calendardate,
        ROUND(revenue / 1e9, 2) as revenue_billions,
        ROUND(netinc / 1e9, 2) as net_income_billions,
        ROUND(marketcap / 1e9, 2) as market_cap_billions,
        ROUND(pe, 1) as pe_ratio,
        ROUND(pb, 1) as pb_ratio,
        ROUND(netinc / revenue * 100, 1) as profit_margin_pct
    FROM latest_financials
    WHERE rn = 1
    AND revenue > 0
    AND marketcap > 10e9  -- Market cap > $10B
    ORDER BY market_cap_billions DESC
    LIMIT 10
""")

print(financial_ratios.to_string(index=False))

In [ ]:
# Visualize P/E ratios vs Profit margins
if len(financial_ratios) > 0:
    fig = px.scatter(
        financial_ratios, 
        x='profit_margin_pct', 
        y='pe_ratio',
        size='market_cap_billions',
        hover_data=['ticker', 'name', 'revenue_billions'],
        title='Technology Sector: P/E Ratio vs Profit Margin',
        labels={
            'profit_margin_pct': 'Profit Margin (%)',
            'pe_ratio': 'P/E Ratio',
            'market_cap_billions': 'Market Cap ($B)'
        }
    )
    
    fig.update_layout(height=600)
    fig.show()
else:
    print("No fundamental data available for visualization")

## 📊 Advanced DuckDB Features

DuckDB has some powerful features for analytical queries. Let's explore a few advanced examples.

In [ ]:
# Example 7: Time series analysis with LAG and LEAD functions
print("📈 Price Momentum Analysis (Last 60 Days):")
momentum_analysis = client.query("""
    WITH price_changes AS (
        SELECT 
            ticker,
            date,
            close,
            LAG(close, 1) OVER (PARTITION BY ticker ORDER BY date) as prev_close,
            LAG(close, 5) OVER (PARTITION BY ticker ORDER BY date) as close_5d_ago,
            LAG(close, 20) OVER (PARTITION BY ticker ORDER BY date) as close_20d_ago
        FROM ndl.sep
        WHERE ticker IN ('AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA')
        AND date >= CURRENT_DATE - INTERVAL '90 days'
    ),
    momentum_calc AS (
        SELECT 
            ticker,
            date,
            close,
            ROUND((close - prev_close) / prev_close * 100, 2) as daily_return,
            ROUND((close - close_5d_ago) / close_5d_ago * 100, 2) as return_5d,
            ROUND((close - close_20d_ago) / close_20d_ago * 100, 2) as return_20d
        FROM price_changes
        WHERE prev_close IS NOT NULL
        AND close_5d_ago IS NOT NULL 
        AND close_20d_ago IS NOT NULL
    )
    SELECT 
        ticker,
        MAX(date) as latest_date,
        ROUND(AVG(daily_return), 2) as avg_daily_return,
        ROUND(STDDEV(daily_return), 2) as volatility,
        ROUND(LAST_VALUE(return_5d) IGNORE NULLS, 2) as latest_5d_return,
        ROUND(LAST_VALUE(return_20d) IGNORE NULLS, 2) as latest_20d_return
    FROM momentum_calc
    GROUP BY ticker
    ORDER BY latest_20d_return DESC
""")

print(momentum_analysis.to_string(index=False))

In [ ]:
# Example 8: Quantile analysis using DuckDB's QUANTILE function
print("📊 Stock Price Distribution Analysis:")
price_quantiles = client.query("""
    SELECT 
        t.sector,
        COUNT(DISTINCT s.ticker) as num_stocks,
        ROUND(QUANTILE(s.close, 0.1), 2) as p10_price,
        ROUND(QUANTILE(s.close, 0.25), 2) as p25_price,
        ROUND(QUANTILE(s.close, 0.5), 2) as median_price,
        ROUND(QUANTILE(s.close, 0.75), 2) as p75_price,
        ROUND(QUANTILE(s.close, 0.9), 2) as p90_price
    FROM ndl.sep s
    JOIN ndl.tickers t ON s.ticker = t.ticker
    WHERE s.date = (SELECT MAX(date) FROM ndl.sep)
    AND t.isdelisted = 'N'
    AND t.sector IS NOT NULL
    AND s.close > 0
    GROUP BY t.sector
    HAVING COUNT(DISTINCT s.ticker) >= 20
    ORDER BY median_price DESC
""")

print(price_quantiles.to_string(index=False))

## 🎯 Practical Investment Screening

Let's create some practical investment screens using DuckDB.

In [ ]:
# Example 9: Value investing screen
print("💎 Value Investing Screen:")
value_screen = client.query("""
    WITH latest_data AS (
        SELECT 
            sf.ticker,
            t.name,
            t.sector,
            sf.pe,
            sf.pb,
            sf.marketcap,
            sf.revenue,
            sf.netinc,
            ROW_NUMBER() OVER (PARTITION BY sf.ticker ORDER BY sf.calendardate DESC) as rn
        FROM ndl.sf1 sf
        JOIN ndl.tickers t ON sf.ticker = t.ticker
        WHERE sf.dimension = 'ARY'
        AND t.isdelisted = 'N'
        AND sf.calendardate >= '2022-01-01'
    )
    SELECT 
        ticker,
        name,
        sector,
        ROUND(pe, 1) as pe_ratio,
        ROUND(pb, 1) as pb_ratio,
        ROUND(marketcap / 1e9, 1) as market_cap_billions,
        ROUND(netinc / revenue * 100, 1) as profit_margin
    FROM latest_data
    WHERE rn = 1
    AND pe BETWEEN 5 AND 20     -- Reasonable P/E ratio
    AND pb BETWEEN 0.5 AND 3    -- Not overvalued on book value
    AND marketcap > 1e9         -- Market cap > $1B
    AND netinc > 0              -- Profitable
    AND revenue > 0
    ORDER BY pe ASC
    LIMIT 15
""")

print(value_screen.to_string(index=False))

In [ ]:
# Example 10: Growth stock screen
print("🚀 Growth Stock Screen:")
growth_screen = client.query("""
    WITH revenue_growth AS (
        SELECT 
            sf.ticker,
            t.name,
            t.sector,
            sf.calendardate,
            sf.revenue,
            sf.marketcap,
            LAG(sf.revenue, 1) OVER (
                PARTITION BY sf.ticker 
                ORDER BY sf.calendardate
            ) as prev_revenue,
            ROW_NUMBER() OVER (
                PARTITION BY sf.ticker 
                ORDER BY sf.calendardate DESC
            ) as rn
        FROM ndl.sf1 sf
        JOIN ndl.tickers t ON sf.ticker = t.ticker
        WHERE sf.dimension = 'ARY'
        AND t.isdelisted = 'N'
        AND sf.calendardate >= '2020-01-01'
        AND sf.revenue > 0
    )
    SELECT 
        ticker,
        name,
        sector,
        ROUND(revenue / 1e9, 2) as latest_revenue_billions,
        ROUND((revenue - prev_revenue) / prev_revenue * 100, 1) as revenue_growth_pct,
        ROUND(marketcap / 1e9, 1) as market_cap_billions
    FROM revenue_growth
    WHERE rn = 1
    AND prev_revenue IS NOT NULL
    AND revenue > prev_revenue   -- Growing revenue
    AND (revenue - prev_revenue) / prev_revenue > 0.15  -- >15% growth
    AND marketcap > 1e9         -- Market cap > $1B
    ORDER BY revenue_growth_pct DESC
    LIMIT 15
""")

print(growth_screen.to_string(index=False))

## 📚 Additional Resources & Next Steps

### 🔗 Useful Links

- **DuckDB Documentation**: [https://duckdb.org/docs/](https://duckdb.org/docs/)
- **SQL Tutorial**: [https://www.sqltutorial.org/](https://www.sqltutorial.org/)
- **Pandas Documentation**: [https://pandas.pydata.org/docs/](https://pandas.pydata.org/docs/)
- **Rice Business Data Portal**: [https://data-portal.rice-business.org](https://data-portal.rice-business.org)

### 💡 Tips for Further Analysis

1. **Time Series Analysis**: Use DuckDB's window functions for rolling calculations
2. **Portfolio Construction**: Combine multiple stocks and calculate correlations
3. **Risk Analysis**: Calculate volatility, beta, and Value at Risk (VaR)
4. **Backtesting**: Test investment strategies using historical data
5. **Sector Rotation**: Analyze sector performance over time

### 🎓 Assignment Ideas

- Build a momentum strategy using moving averages
- Create a diversified portfolio across sectors
- Analyze the relationship between volume and price movements
- Study earnings announcements and stock price reactions
- Compare growth vs value investing performance

---

**Happy analyzing! 📊🚀**

In [ ]:
# Summary: Your exploration summary
print("🎉 Rice Business Stock Market Data Analysis Complete!")
print("\n📊 What we covered:")
print("   ✅ Connected to Rice Business data with access token")
print("   ✅ Explored available data tables and statistics")
print("   ✅ Performed basic and advanced DuckDB queries")
print("   ✅ Analyzed sector performance and stock momentum")
print("   ✅ Explored fundamental analysis with financial ratios")
print("   ✅ Created practical investment screens")
print("\n💡 Ready to dive deeper into financial analysis!")
print("\n🔗 Don't forget to get your own access token from the Rice Business Data Portal")